# DAT210x - Programming with Python for DS

## Module5- Lab5

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

matplotlib.style.use('ggplot') # Look Pretty

### A Convenience Function

In [42]:
def plotDecisionBoundary(model, X, y):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    padding = 0.6
    resolution = 0.0025
    colors = ['royalblue','forestgreen','ghostwhite']

    # Calculate the boundaris
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()
    x_range = x_max - x_min
    y_range = y_max - y_min
    x_min -= x_range * padding
    y_min -= y_range * padding
    x_max += x_range * padding
    y_max += y_range * padding

    # Create a 2D Grid Matrix. The values stored in the matrix
    # are the predictions of the class at at said location
    xx, yy = np.meshgrid(np.arange(x_min, x_max, resolution),
                       np.arange(y_min, y_max, resolution))

    # What class does the classifier say?
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the contour map
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.terrain)

    # Plot the test original points as well...
    for label in range(len(np.unique(y))):
        indices = np.where(y == label)
        plt.scatter(X[indices, 0], X[indices, 1], c=colors[label], label=str(label), alpha=0.8)

    p = model.get_params()
    plt.axis('tight')
    plt.title('K = ' + str(p['n_neighbors']))

### The Assignment

Load up the dataset into a variable called `X`. Check `.head` and `dtypes` to make sure you're loading your data properly--don't fail on the 1st step!

In [65]:
# .. your code here ..
x = pd.read_csv('Datasets/wheat.data',index_col=0)
x.head()
# x.dtypes

,area,perimeter,compactness,length,width,asymmetry,groove,wheat_type
id,,,,,,,,
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,kama
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,kama
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,kama
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,kama
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,kama


Copy the `wheat_type` series slice out of `X`, and into a series called `y`. Then drop the original `wheat_type` column from the `X`:

In [66]:
y = x['wheat_type']
x = x.drop(['wheat_type'],axis=1)

Do a quick, "ordinal" conversion of `y`. In actuality our classification isn't ordinal, but just as an experiment...

In [67]:
# .. your code here ..
y.astype('category',ordered=True)

id
0          kama
1          kama
2          kama
3          kama
4          kama
5          kama
6          kama
7      canadian
8          kama
9          kama
10         kama
11         kama
12         kama
13         kama
14         kama
15         kama
16         kama
17         kama
18         kama
19         kama
20         kama
21         kama
22         kama
23         kama
24         kama
25         kama
26         kama
27         kama
28         kama
29         kama
         ...   
180    canadian
181    canadian
182    canadian
183    canadian
184    canadian
185    canadian
186    canadian
187    canadian
188    canadian
189    canadian
190    canadian
191    canadian
192    canadian
193    canadian
194    canadian
195    canadian
196    canadian
197    canadian
198    canadian
199    canadian
200    canadian
201    canadian
202    canadian
203    canadian
204    canadian
205    canadian
206    canadian
207    canadian
208    canadian
209    canadian
Name: wheat_type, dty

Do some basic nan munging. Fill each row's nans with the mean of the feature:

In [68]:
# .. your code here ..
# area	perimeter	compactness	length	width	asymmetry	groove
# x.area.fillna( df.area.mean() )
# x.perimeter.fillna( df.perimeter.mean() )
# x.compactness.fillna( df.compactness.mean() )
# x.length.fillna( df.length.mean() )
# x.width.fillna( df.width.mean() )
# x.asymmetry.fillna( df.asymmetry.mean() )
# x.groove.fillna( df.groove.mean() )
x.fillna(x.mean(),inplace=True)

,area,perimeter,compactness,length,width,asymmetry,groove
id,,,,,,,
0,15.26,14.84,0.8710,5.763,3.312,2.2210,5.220000
1,14.88,14.57,0.8811,5.554,3.333,1.0180,4.956000
2,14.29,14.09,0.9050,5.291,3.337,2.6990,4.825000
3,13.84,13.94,0.8955,5.324,3.379,2.2590,4.805000
4,16.14,14.99,0.9034,5.658,3.562,1.3550,5.175000
5,14.38,14.21,0.8951,5.386,3.312,2.4620,4.956000
6,14.69,14.49,0.8799,5.563,3.259,3.5860,5.219000
7,14.11,14.10,0.8911,5.420,3.302,2.7000,5.407529
8,16.63,15.46,0.8747,6.053,3.465,2.0400,5.877000


Split `X` into training and testing data sets using `train_test_split()`. Use `0.33` test size, and use `random_state=1`. This is important so that your answers are verifiable. In the real world, you wouldn't specify a random_state:

In [113]:
# .. your code here ..
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)

Create an instance of SKLearn's Normalizer class and then train it using its .fit() method against your _training_ data. The reason you only fit against your training data is because in a real-world situation, you'll only have your training data to train with! In this lab setting, you have both train+test data; but in the wild, you'll only have your training data, and then unlabeled data you want to apply your models to.

In [114]:
# .. your code here ..
from sklearn.preprocessing import Normalizer
trainNml = Normalizer().fit(x_train)
trainNml

Normalizer(copy=True, norm='l2')

With your trained pre-processor, transform both your training AND testing data. Any testing data has to be transformed with your preprocessor that has ben fit against your training data, so that it exist in the same feature-space as the original data used to train your models.

In [115]:
# .. your code here ..
trainTnml = Normalizer.transform(trainNml, x_train)
testTnml = Normalizer.transform(trainNml, x_test)

Just like your preprocessing transformation, create a PCA transformation as well. Fit it against your training data, and then project your training and testing features into PCA space using the PCA model's `.transform()` method. This has to be done because the only way to visualize the decision boundary in 2D would be if your KNN algo ran in 2D as well:

In [121]:
# .. your code here ..
from sklearn.decomposition import PCA
pca = PCA(n_components=2, svd_solver='full')
pca.fit(x_train)

trainTpca = pca.transform(x_train)
testTpca = pca.transform(x_test)

Create and train a KNeighborsClassifier. Start with `K=9` neighbors. Be sure train your classifier against the pre-processed, PCA- transformed training data above! You do not, of course, need to transform your labels.

In [122]:
# .. your code here ..
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=9)
knn = knn.fit(x_train, y_train)

In [125]:
# I hope your KNeighbors classifier model from earlier was named 'knn'
# If not, adjust the following line:
plotDecisionBoundary(knn, trainTnml, testTnml)

ValueError: query data dimension must match training data dimension

Display the accuracy score of your test data/labels, computed by your KNeighbors model. You do NOT have to run `.predict` before calling `.score`, since `.score` will take care of running your predictions for you automatically.

In [139]:
# .. your code here ..
knn = KNeighborsClassifier(n_neighbors=1)
knn = knn.fit(x_train, y_train)

knn.score(x_test,y_test)

0.87142857142857144

### Bonus

Instead of the ordinal conversion, try and get this assignment working with a proper Pandas get_dummies for feature encoding. You might have to update some of the `plotDecisionBoundary()` code.

In [ ]:
plt.show()